## Libraries

In [38]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import emoji
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,Dropout,Bidirectional
from tensorflow.keras.utils import pad_sequences 
from tensorflow.keras.preprocessing.text import Tokenizer 

## Data preprocessing

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kasir\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df=pd.read_csv('reviews_data.csv')
df

,name,location,Date,Rating,Review,Image_Links
0,Helen,"Wichita Falls, TX","Reviewed Sept. 13, 2023",5.0,Amber and LaDonna at the Starbucks on Southwes...,['No Images']
1,Courtney,"Apopka, FL","Reviewed July 16, 2023",5.0,** at the Starbucks by the fire station on 436...,['No Images']
2,Daynelle,"Cranberry Twp, PA","Reviewed July 5, 2023",5.0,I just wanted to go out of my way to recognize...,['https://media.consumeraffairs.com/files/cach...
3,Taylor,"Seattle, WA","Reviewed May 26, 2023",5.0,Me and my friend were at Starbucks and my card...,['No Images']
4,Tenessa,"Gresham, OR","Reviewed Jan. 22, 2023",5.0,I’m on this kick of drinking 5 cups of warm wa...,['https://media.consumeraffairs.com/files/cach...
...,...,...,...,...,...,...
845,Becky,"Agoura Hills, CA","Reviewed July 13, 2006",NaN,I ordered two venti frappacino's without whipp...,['No Images']
846,Bob,"Goodrich, MI","Reviewed Jan. 3, 2005",NaN,No Review Text,['No Images']
847,Erik,"Valley Village, CA","Reviewed Nov. 5, 2004",NaN,"DEMANDED TIPS FROM ME, THEN MADE ME WAIT UNTIL...",['No Images']
848,Andrew,"Fallbrook, CA","Reviewed Oct. 20, 2004",NaN,No Review Text,['No Images']


In [4]:
df.drop('Image_Links',inplace=True,axis=1)
df.head()

,name,location,Date,Rating,Review
0,Helen,"Wichita Falls, TX","Reviewed Sept. 13, 2023",5.0,Amber and LaDonna at the Starbucks on Southwes...
1,Courtney,"Apopka, FL","Reviewed July 16, 2023",5.0,** at the Starbucks by the fire station on 436...
2,Daynelle,"Cranberry Twp, PA","Reviewed July 5, 2023",5.0,I just wanted to go out of my way to recognize...
3,Taylor,"Seattle, WA","Reviewed May 26, 2023",5.0,Me and my friend were at Starbucks and my card...
4,Tenessa,"Gresham, OR","Reviewed Jan. 22, 2023",5.0,I’m on this kick of drinking 5 cups of warm wa...


In [5]:
df1=df[df['Review']!='No Review Text']
df1

,name,location,Date,Rating,Review
0,Helen,"Wichita Falls, TX","Reviewed Sept. 13, 2023",5.0,Amber and LaDonna at the Starbucks on Southwes...
1,Courtney,"Apopka, FL","Reviewed July 16, 2023",5.0,** at the Starbucks by the fire station on 436...
2,Daynelle,"Cranberry Twp, PA","Reviewed July 5, 2023",5.0,I just wanted to go out of my way to recognize...
3,Taylor,"Seattle, WA","Reviewed May 26, 2023",5.0,Me and my friend were at Starbucks and my card...
4,Tenessa,"Gresham, OR","Reviewed Jan. 22, 2023",5.0,I’m on this kick of drinking 5 cups of warm wa...
...,...,...,...,...,...
841,Mary,"Hillsboro, OR","Reviewed Dec. 16, 2007",NaN,"When using my debit card, how much and for how..."
842,Angie,"Innsbruck, OR","Reviewed Sept. 17, 2007",NaN,I just bought a starbucks thermos mug about a ...
844,Sabine,"Redondo Beach, CA","Reviewed Dec. 19, 2006",NaN,I'm (was) a regular customer who ordered a lat...
845,Becky,"Agoura Hills, CA","Reviewed July 13, 2006",NaN,I ordered two venti frappacino's without whipp...


In [6]:
df1['Rating'].value_counts()

1.0    450
2.0     98
5.0     83
4.0     39
3.0     33
Name: Rating, dtype: int64

In [7]:
df1.isnull().sum()

name          0
location      0
Date          0
Rating      110
Review        0
dtype: int64

In [8]:
dftest1=df1[df1.isnull().any(axis=1)]
dftest1

,name,location,Date,Rating,Review
704,James,"Kansas City, MO","Reviewed July 25, 2011",NaN,I just wanted to amend my email the I sent to ...
705,James,"Kansas City, MO","Reviewed July 25, 2011",NaN,"Recently, I have gone to your Starbucks at Bar..."
706,Mike,"Revere, ma","Reviewed June 26, 2011",NaN,Upon my first visit to this location on my way...
707,Hughes,"Macclesfield, Other","Reviewed Jan. 13, 2011",NaN,"Recently, British Royal Marines in Iraq wrote ..."
708,Sherrilynn,"Jenison, MI","Reviewed Jan. 4, 2011",NaN,"On the way to catch our plane, we got a medium..."
...,...,...,...,...,...
841,Mary,"Hillsboro, OR","Reviewed Dec. 16, 2007",NaN,"When using my debit card, how much and for how..."
842,Angie,"Innsbruck, OR","Reviewed Sept. 17, 2007",NaN,I just bought a starbucks thermos mug about a ...
844,Sabine,"Redondo Beach, CA","Reviewed Dec. 19, 2006",NaN,I'm (was) a regular customer who ordered a lat...
845,Becky,"Agoura Hills, CA","Reviewed July 13, 2006",NaN,I ordered two venti frappacino's without whipp...


In [9]:
txtdf=df1[['Review','Rating']]
txtdf.head()

,Review,Rating
0,Amber and LaDonna at the Starbucks on Southwes...,5.0
1,** at the Starbucks by the fire station on 436...,5.0
2,I just wanted to go out of my way to recognize...,5.0
3,Me and my friend were at Starbucks and my card...,5.0
4,I’m on this kick of drinking 5 cups of warm wa...,5.0


In [10]:
def clean(text):
    text = re.sub('[^a-zA-Z]', ' ', str(text).lower().strip())
    text = re.sub('@[A-Za-z0-9_]+', '', text) 
    text = re.sub('#','',text) 
    text = re.sub('RT[\s]+','',text)
    text = re.sub('https?:\/\/\S+', '', text) 
    text = re.sub('\n',' ',text)
    text = emoji.replace_emoji(text, replace='')
    return text

In [11]:
txtdf['Review']=txtdf['Review'].apply(clean)

C:\Users\kasir\AppData\Local\Temp\ipykernel_16424\2520822112.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  txtdf['Review']=txtdf['Review'].apply(clean)


In [12]:
txtdf

,Review,Rating
0,amber and ladonna at the starbucks on southwes...,5.0
1,at the starbucks by the fire station on ...,5.0
2,i just wanted to go out of my way to recognize...,5.0
3,me and my friend were at starbucks and my card...,5.0
4,i m on this kick of drinking cups of warm wa...,5.0
...,...,...
841,when using my debit card how much and for how...,NaN
842,i just bought a starbucks thermos mug about a ...,NaN
844,i m was a regular customer who ordered a lat...,NaN
845,i ordered two venti frappacino s without whipp...,NaN


In [13]:
txtdf.dropna(inplace=True)
txtdf

C:\Users\kasir\AppData\Local\Temp\ipykernel_16424\2392683827.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  txtdf.dropna(inplace=True)


,Review,Rating
0,amber and ladonna at the starbucks on southwes...,5.0
1,at the starbucks by the fire station on ...,5.0
2,i just wanted to go out of my way to recognize...,5.0
3,me and my friend were at starbucks and my card...,5.0
4,i m on this kick of drinking cups of warm wa...,5.0
...,...,...
700,i ordered via starbucks coffee online i recei...,1.0
701,my name is ric i am journalist by professi...,3.0
702,the bagel was ice cold not cut and not toasted,1.0
703,in the morning of monday august at ...,1.0


In [14]:
txtdf.reset_index(inplace=True)

In [15]:
le=LabelEncoder()
txtdf['Rating'] = le.fit_transform(txtdf['Rating'])

C:\Users\kasir\AppData\Local\Temp\ipykernel_16424\1006121447.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  txtdf['Rating'] = le.fit_transform(txtdf['Rating'])


In [16]:
lem = WordNetLemmatizer()
corpus=[]
for i in range(0, len(txtdf)):
    review = txtdf['Review'][i]
    review = word_tokenize(review)
    review = [lem.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    txtdf['Review'][i] = review
    corpus.append(review)
txtdf['Review'][175]

C:\Users\kasir\AppData\Local\Temp\ipykernel_16424\1466972264.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  txtdf['Review'][i] = review


'charge chai tea latte train employee make store make way horrific experience today starbucks riverside ca literally give boiling milk asked add tea lady store asked want syrup hell customer know using secret syrup make chai tea latte repeatedly asked need tea course got angry walk away boiling milk first make worst chai tea latte still drinking make worst exploit customer like hopefully somebody see take necessary action'

In [17]:
def count_words(text):
    tokens = word_tokenize(text)  # Tokenize the text
    return len(tokens)
totwords = txtdf['Review'].apply(count_words).sum()
totwords

30641

In [18]:
X = txtdf['Review']
Y = txtdf['Rating']

In [19]:
Y

0      4
1      4
2      4
3      4
4      4
      ..
698    0
699    2
700    0
701    0
702    4
Name: Rating, Length: 703, dtype: int64

In [20]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=42)

In [21]:
x_train

294    never get drink wrong employee friendly profes...
210    sure distribution center starbucks north carol...
619    balance showing card every time looked quite t...
507    everyday morning drink dark roast coffee starb...
404    week ago mother like every morning go starbuck...
                             ...                        
71     visited starbucks flowood mississippi may youn...
106    sept around pm starbucks sherway garden toront...
270    expensive good drink like food big variety ple...
435    buy smoothy different place time change bought...
102    could give corporate office star would wheneve...
Name: Review, Length: 597, dtype: object

 TfidfVectorizer

In [22]:
tfidfvec = TfidfVectorizer()

In [23]:
tfidf_matrix = tfidfvec.fit_transform(corpus)

In [24]:
xtrtfidf = tfidfvec.fit_transform(x_train)
xtsttfidf = tfidfvec.transform(x_test)

## XGBClassifier

In [25]:
cls = XGBClassifier()
cls.fit(xtrtfidf, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', ...)

In [26]:
xtsttfidf = tfidfvec.transform(x_test)
ypred = cls.predict(xtsttfidf)

In [27]:
accuracy = accuracy_score(y_test, ypred)
report = classification_report(y_test, ypred)

C:\Users\kasir\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kasir\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kasir\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
print("accuracy:", accuracy)
print("classification report:\n", report)

Accuracy: 0.6132075471698113
Classification Report:
               precision    recall  f1-score   support

           0       0.66      0.95      0.78        61
           1       0.00      0.00      0.00        17
           2       0.00      0.00      0.00         6
           3       0.00      0.00      0.00         7
           4       0.64      0.47      0.54        15

    accuracy                           0.61       106
   macro avg       0.26      0.28      0.26       106
weighted avg       0.47      0.61      0.52       106



## LSTM model

In [29]:
tokenizer = Tokenizer(num_words=totwords, oov_token='<OOV>') 
tokenizer.fit_on_texts(x_train) 

In [30]:
train_seq = tokenizer.texts_to_sequences(x_train) 
test_seq = tokenizer.texts_to_sequences(x_test) 
  
train_pad = pad_sequences(train_seq, 
                          maxlen=40, 
                          truncating="post", 
                          padding="post") 
test_pad = pad_sequences(test_seq, 
                         maxlen=40, 
                         truncating="post", 
                         padding="post") 

In [36]:
model=Sequential()
model.add(Embedding(input_dim=totwords,output_dim=64))
model.add(LSTM(128,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dense(5, activation="softmax")) 
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 64)          1961024   
                                                                 
 lstm_3 (LSTM)               (None, None, 128)         98816     
                                                                 
 dropout_1 (Dropout)         (None, None, 128)         0         
                                                                 
 lstm_4 (LSTM)               (None, 64)                49408     
                                                                 
 dense_2 (Dense)             (None, 5)                 325       
                                                                 
Total params: 2,109,573
Trainable params: 2,109,573
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.compile("RMSprop","sparse_categorical_crossentropy",metrics=["accuracy"]) 
history = model.fit(train_pad,y_train,epochs=10) 

Epoch 1/10
19/19 [==============================] - 10s 162ms/step - loss: 1.2158 - accuracy: 0.6231
Epoch 2/10
19/19 [==============================] - 3s 150ms/step - loss: 1.0410 - accuracy: 0.6683
Epoch 3/10
19/19 [==============================] - 3s 159ms/step - loss: 0.9348 - accuracy: 0.6985
Epoch 4/10
19/19 [==============================] - 3s 161ms/step - loss: 0.7514 - accuracy: 0.7404
Epoch 5/10
19/19 [==============================] - 3s 172ms/step - loss: 0.6618 - accuracy: 0.7504
Epoch 6/10
19/19 [==============================] - 3s 173ms/step - loss: 0.5871 - accuracy: 0.7672
Epoch 7/10
19/19 [==============================] - 3s 166ms/step - loss: 0.4688 - accuracy: 0.8224
Epoch 8/10
19/19 [==============================] - 3s 176ms/step - loss: 0.4707 - accuracy: 0.8375
Epoch 9/10
19/19 [==============================] - 3s 167ms/step - loss: 0.4225 - accuracy: 0.8375
Epoch 10/10
19/19 [==============================] - 3s 171ms/step - loss: 0.3446 - accuracy: 0.869

## Biderectional GRU model

In [40]:
model1=Sequential()
model1.add(Embedding(input_dim=totwords,output_dim=64))
model1.add(Bidirectional(GRU(64, return_sequences=True))) 
model1.add(Bidirectional(GRU(64))) 
model1.add(Dense(128, activation="relu")) 
model1.add(Dropout(0.4)) 
model1.add(Dense(5, activation="softmax")) 
model1.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 64)          1961024   
                                                                 
 bidirectional_2 (Bidirectio  (None, None, 128)        49920     
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                                                 
 dense_5 (Dense)             (None, 128)               16512     
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 5)                

In [41]:
model1.compile("RMSprop","sparse_categorical_crossentropy",metrics=["accuracy"]) 
history = model1.fit(train_pad,y_train,epochs=10) 

Epoch 1/10
19/19 [==============================] - 16s 173ms/step - loss: 1.2736 - accuracy: 0.6265
Epoch 2/10
19/19 [==============================] - 3s 149ms/step - loss: 1.0711 - accuracy: 0.6516
Epoch 3/10
19/19 [==============================] - 3s 153ms/step - loss: 0.9746 - accuracy: 0.6583
Epoch 4/10
19/19 [==============================] - 3s 150ms/step - loss: 0.7966 - accuracy: 0.7219
Epoch 5/10
19/19 [==============================] - 3s 153ms/step - loss: 0.6290 - accuracy: 0.7487
Epoch 6/10
19/19 [==============================] - 3s 149ms/step - loss: 0.4875 - accuracy: 0.8074
Epoch 7/10
19/19 [==============================] - 3s 145ms/step - loss: 0.4250 - accuracy: 0.8174
Epoch 8/10
19/19 [==============================] - 3s 144ms/step - loss: 0.3560 - accuracy: 0.8710
Epoch 9/10
19/19 [==============================] - 3s 146ms/step - loss: 0.2708 - accuracy: 0.8777
Epoch 10/10
19/19 [==============================] - 3s 147ms/step - loss: 0.3009 - accuracy: 0.879